In [1]:
import pandas as pd
import numpy as np
import io

# --- 加载数据 ---
# 请将下面的StringIO替换为你的文件名，例如 'train.csv'
train_df = pd.read_csv('data/Antai_hackathon_train.csv')
attr_df = pd.read_csv('data/Antai_hackathon_attr.csv')

# 检查数据加载情况
print("--- 原始训练数据 (train.csv) ---")
print(train_df.head())
print("\n--- 原始商品属性数据 (item_attr.csv) ---")
print(attr_df.head())

merged_df = pd.merge(train_df, attr_df, on='item_id', how='left')

print(train_df.shape)
print(attr_df.shape)

# 显示合并后的表格信息
print("--- 1. 数据合并结果 ---")
print("合并后表格的维度:", merged_df.shape)
print("合并后表格的前5行:")
print(merged_df.head())
print("\n")

--- 原始训练数据 (train.csv) ---
  buyer_country_id  buyer_admin_id  item_id    create_order_time  irank
0               xx          489859        1  2018-04-11 03:28:02      9
1               xx         2567786        2  2018-04-14 08:24:26     26
2               xx         3408746        3  2018-04-17 02:11:56      7
3               xx         2801580        4  2018-04-20 10:11:17      3
4               xx         1348149        5  2018-04-17 10:49:05      4

--- 原始商品属性数据 (item_attr.csv) ---
   item_id  cate_id  store_id  item_price
0   140446     1413     11822           1
1   403593     2313     19712           1
2   252621     1682      6622           1
3   204530     1413     11822           1
4   340076      181     24403           1
(6989817, 5)
(1924269, 4)
--- 1. 数据合并结果 ---
合并后表格的维度: (6989817, 8)
合并后表格的前5行:
  buyer_country_id  buyer_admin_id  item_id    create_order_time  irank  \
0               xx          489859        1  2018-04-11 03:28:02      9   
1               xx         

In [5]:
merged_df.groupby('buyer_admin_id').head()

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,cate_id,store_id,item_price
0,xx,489859,1,2018-04-11 03:28:02,9,2228.0,9694.0,4491.0
1,xx,2567786,2,2018-04-14 08:24:26,26,3667.0,4364.0,2751.0
2,xx,3408746,3,2018-04-17 02:11:56,7,153.0,8085.0,656.0
3,xx,2801580,4,2018-04-20 10:11:17,3,3359.0,3345.0,2501.0
4,xx,1348149,5,2018-04-17 10:49:05,4,1156.0,1892.0,589.0
...,...,...,...,...,...,...,...,...
6985354,xx,1305397,7580905,2018-04-16 02:33:33,3,2514.0,76810.0,101.0
6985355,xx,1305397,7580905,2018-04-16 02:33:33,2,2514.0,76810.0,101.0
6985356,xx,1305397,7580905,2018-04-16 02:33:33,7,2514.0,76810.0,101.0
6989120,xx,1743518,7585013,2018-04-21 15:28:54,12,320.0,76684.0,81.0


In [7]:
import pandas as pd

# --- 1. 加载数据 ---
# 请确保文件名与你的本地文件一致
try:
    train_df = pd.read_csv('data/Antai_hackathon_train.csv')
    test_df = pd.read_csv('data/dianshang_test.csv')
except FileNotFoundError:
    print("错误：请确保 train.csv 和 test_without_last.csv 文件在当前目录下！")
    # 为了让代码能继续演示，这里使用一个空的DataFrame
    # 在你的环境中，如果文件存在，这部分代码不会执行
    train_df = pd.DataFrame({'item_id': [1, 1, 2, 3, 2, 1]})
    test_df = pd.DataFrame({'buyer_admin_id': [1001, 1002, 1001]})


print("数据加载完成。")
print(f"训练集记录数: {len(train_df)}")
print(f"测试集记录数: {len(test_df)}")


# --- 2. 计算全局商品热度 ---
# 使用 value_counts() 可以非常高效地计算每个 item_id 出现的次数
print("\n正在计算商品热度...")
item_popularity = train_df['item_id'].value_counts()

# 获取热度最高的前30个商品
top_30_items = item_popularity.head(30).index.tolist()

print("Top 30 热门商品计算完成:")
print(top_30_items)


# --- 3. 为测试集用户生成推荐列表并创建提交文件 ---
print("\n正在为测试集用户生成推荐...")
# 获取测试集中所有不重复的用户ID
test_user_ids = test_df['buyer_admin_id'].unique()
print(f"需要为 {len(test_user_ids)} 名测试用户生成预测。")

# 准备一个列表来存储最终的提交结果
submission_data = []

# 将Top 30商品列表转换为字符串，用逗号连接，这样效率更高
top_30_items_str = ",".join(map(str, top_30_items))

for user_id in test_user_ids:
    # 对于全局热度模型，每个用户的推荐列表都是一样的
    # 格式为：[用户ID, 预测商品1, 预测商品2, ...]
    # 为了直接写入CSV，我们构建一行字符串
    submission_data.append([user_id] + top_30_items)

# --- 4. 保存提交文件 ---
# 定义提交文件的列名
columns = ['buyer_admin_id'] + [f'predict {i+1}' for i in range(30)]

# 创建DataFrame
submission_df = pd.DataFrame(submission_data, columns=columns)

# 保存为CSV文件，注意 index=False 参数是必须的，否则会多出一列索引
# 比赛要求的文件名为 username.csv, 请自行修改'global_popularity_baseline.csv'
submission_filename = 'global_popularity_baseline.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n提交文件 '{submission_filename}' 已成功生成！")
print("文件预览:")
print(submission_df.head())

数据加载完成。
训练集记录数: 6989817
测试集记录数: 140380

正在计算商品热度...
Top 30 热门商品计算完成:
[7493101, 516873, 3964264, 7557509, 7482805, 3058048, 426022, 1872817, 413606, 6162363, 648691, 797627, 5421409, 1266501, 584059, 4163903, 928637, 564243, 7538392, 2982899, 7533383, 959326, 34624, 5759221, 560251, 5644811, 5378792, 7324348, 691497, 2548432]

正在为测试集用户生成推荐...
需要为 10576 名测试用户生成预测。

提交文件 'global_popularity_baseline.csv' 已成功生成！
文件预览:
   buyer_admin_id  predict 1  predict 2  predict 3  predict 4  predict 5  \
0         6664334    7493101     516873    3964264    7557509    7482805   
1         6394777    7493101     516873    3964264    7557509    7482805   
2         1746935    7493101     516873    3964264    7557509    7482805   
3         5828988    7493101     516873    3964264    7557509    7482805   
4          421713    7493101     516873    3964264    7557509    7482805   

   predict 6  predict 7  predict 8  predict 9  ...  predict 21  predict 22  \
0    3058048     426022    1872817     413606  .

In [9]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

# --- 1. 加载数据 ---
print("正在加载数据...")
try:
    train_df = pd.read_csv('data/Antai_hackathon_train.csv')
    test_df = pd.read_csv('data/dianshang_test.csv')
except FileNotFoundError:
    print("错误：请确保 train.csv 和 test_.csv 文件在当前目录下！")


# --- 2. 构建商品转移矩阵 ---
print("开始构建商品转移矩阵...")

# 确保时间列是datetime类型，并按用户和时间排序
train_df['create_order_time'] = pd.to_datetime(train_df['create_order_time'])
train_df = train_df.sort_values(by=['buyer_admin_id', 'create_order_time'])

# 使用defaultdict来高效地存储转移关系
# 格式: {item_A: {item_B: count, item_C: count}, ...}
transition_matrix = defaultdict(lambda: defaultdict(int))


正在加载数据...
开始构建商品转移矩阵...


In [12]:

# 使用tqdm来显示进度条
# 我们按用户分组，然后处理每个用户的购买序列
for user_id, user_group in tqdm(train_df.groupby('buyer_admin_id'), desc="Processing users"):
    # 将用户的购买商品ID转换为列表
    item_sequence = user_group['item_id'].tolist()
    # 遍历序列，创建(前一个商品, 后一个商品)的对
    for i in range(len(item_sequence) - 1):
        prev_item = item_sequence[i]
        next_item = item_sequence[i+1]
        transition_matrix[prev_item][next_item] += 1

print("转移矩阵构建完成。")

# --- 3. 准备回退策略：全局热度榜单 ---
print("正在计算全局热度榜单 (用于回退)...")
item_popularity = train_df['item_id'].value_counts()
global_top_30_items = item_popularity.head(30).index.tolist()


Processing users: 100%|██████████| 483117/483117 [00:16<00:00, 29535.23it/s]


转移矩阵构建完成。
正在计算全局热度榜单 (用于回退)...


In [36]:

# --- 4. 为测试用户生成推荐 ---
print("正在为测试用户生成推荐列表...")

# 找到每个测试用户的最后一次购买记录
test_df['create_order_time'] = pd.to_datetime(test_df['create_order_time'])
last_purchases = test_df.sort_values('create_order_time').drop_duplicates('buyer_admin_id', keep='last')
user_last_item_map = dict(zip(last_purchases['buyer_admin_id'], last_purchases['item_id']))
all_test_users = test_df['buyer_admin_id'].unique()



正在为测试用户生成推荐列表...


In [38]:
submission_data = []

for user_id in tqdm(all_test_users, desc="Generating predictions"):
    recommendations = []
    
    # 获取该用户的最后购买商品
    last_item = user_last_item_map.get(user_id)
    
    # 如果该用户有最后购买记录，并且该商品在我们的转移矩阵中
    if last_item and last_item in transition_matrix:
        # 从转移矩阵中获取候选推荐
        candidates = transition_matrix[last_item]
        # 按购买次数降序排序
        sorted_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)
        # 提取商品ID
        recommendations = [item_id for item_id, count in sorted_candidates]

    # --- 回退和补充逻辑 ---
    # 如果个性化推荐不足30个，或根本没有个性化推荐
    if len(recommendations) < 30:
        # 使用全局热度榜单来补充
        # 创建一个已推荐商品的集合，用于快速去重
        rec_set = set(recommendations)
        for popular_item in global_top_30_items:
            if popular_item not in rec_set:
                recommendations.append(popular_item)
            # 当推荐列表达到30个时，停止补充
            if len(recommendations) >= 30:
                break
    
    # 截取最终的Top 30
    final_recommendations = recommendations[:30]
    submission_data.append([user_id] + final_recommendations)

# --- 5. 保存提交文件 ---
columns = ['buyer_admin_id'] + [f'predict {i+1}' for i in range(30)]
submission_df = pd.DataFrame(submission_data, columns=columns)

submission_filename = 'item_cf_baseline.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n提交文件 '{submission_filename}' 已成功生成！")
print("文件预览:")
print(submission_df.head())

Generating predictions: 100%|██████████| 10576/10576 [00:01<00:00, 7915.22it/s]



提交文件 'item_cf_baseline.csv' 已成功生成！
文件预览:
   buyer_admin_id  predict 1  predict 2  predict 3  predict 4  predict 5  \
0         6664334    5736363    5881751    4091904     158258    2695682   
1         6394777    4430243    1642080    2391712    7493101     516873   
2         1746935    7493101     516873    3964264    7557509    7482805   
3         5828988     926095    7493101     516873    3964264    7557509   
4          421713    3517157     828371    4105328    3537214    3812200   

   predict 6  predict 7  predict 8  predict 9  ...  predict 21  predict 22  \
0    6886784    4435110    6207830    1305003  ...     3964264     7557509   
1    3964264    7557509    7482805    3058048  ...      564243     7538392   
2    3058048     426022    1872817     413606  ...     7533383      959326   
3    7482805    3058048     426022    1872817  ...     2982899     7533383   
4    6831580    4883526    4120858    2336195  ...     5594492     3555140   

   predict 23  predict 24  predi

In [ ]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import math
from tqdm import tqdm

# --- 数据加载 ---
print("正在加载数据...")
try:
    train_df = pd.read_csv('data/Antai_hackathon_train.csv')
    test_df = pd.read_csv('data/dianshang_test.csv')
except FileNotFoundError:
    print("错误：请确保 train.csv 和 test_without_last.csv 文件在当前目录下！")
    # 模拟数据
    train_df = pd.DataFrame({'buyer_admin_id': [1, 1, 1, 2, 2, 3, 3, 3], 'item_id': [10, 20, 30, 10, 40, 50, 10, 20]})
    test_df = pd.DataFrame({'buyer_admin_id': [1, 2, 3], 'item_id': [30, 40, 20]})

# --------------------------------------------------------------------------------------
# 步骤1: 构建以用户为行、物品为列的共现矩阵 (的稀疏表示)
# --------------------------------------------------------------------------------------
# 我们不创建真实的m x n矩阵，而是创建一个字典，key是用户ID，value是该用户购买过的物品集合。
# 这在功能上等价于存储了巨大矩阵中所有值为1的位置。
print("步骤1: 正在构建用户-物品关系的稀疏表示...")
user_items_dict_train = train_df.groupby('buyer_admin_id')['item_id'].apply(set).to_dict()
print("用户-物品关系构建完成。")


# --------------------------------------------------------------------------------------
# 步骤2: 计算共现矩阵两两列间的相似性
# --------------------------------------------------------------------------------------
# 计算两列(item_i, item_j)的相似度，等价于计算 sim(i, j) = cos(vec_i, vec_j)
# cos(vec_i, vec_j) = (vec_i · vec_j) / (||vec_i|| * ||vec_j||)
# 其中：
#   - vec_i · vec_j (点积) = 购买了i和j的共同用户数
#   - ||vec_i|| (L2范数) = sqrt(购买了i的用户数)
# 我们下面的代码就是为了计算这两个核心指标。

print("步骤2: 正在计算物品间的相似度...")

# a. 计算点积 (dot product)，即分子部分：共同购买的用户数
co_occurrence_matrix = defaultdict(lambda: defaultdict(int))
# b. 计算范数 (norm) 的平方，即分母部分：每个物品的购买用户数
item_popularity = defaultdict(int)

# 通过遍历我们稀疏存储的用户-物品关系，来高效地完成计算
for user, items in tqdm(user_items_dict_train.items(), desc="计算共现次数与物品热度"):
    for item in items:
        # 每当一个物品在一个用户的集合里，它的购买用户数就+1
        item_popularity[item] += 1
    # 对于该用户购买过的每一对物品(i, j)，它们的共同购买用户数就+1
    for i, j in combinations(items, 2):
        co_occurrence_matrix[i][j] += 1
        co_occurrence_matrix[j][i] += 1

# c. 组合起来，计算最终的余弦相似度
item_similarity_matrix = defaultdict(lambda: defaultdict(float))
for i, related_items in tqdm(co_occurrence_matrix.items(), desc="计算最终的余弦相似度"):
    for j, count in related_items.items():
        # 这就是余弦相似度公式的直接应用
        # count = vec_i · vec_j
        # item_popularity[i] = ||vec_i||^2
        # item_popularity[j] = ||vec_j||^2
        similarity = count / math.sqrt(item_popularity[i] * item_popularity[j])
        item_similarity_matrix[i][j] = similarity

print("物品相似度矩阵构建完成。")


# --------------------------------------------------------------------------------------
# 步骤3 & 4: 获取用户正反馈物品，找出相似TopK，排序生成推荐列表
# --------------------------------------------------------------------------------------
print("步骤3 & 4: 正在为用户生成推荐列表...")

# 准备回退策略：全局热度榜单
global_top_items_sorted = sorted(item_popularity.items(), key=lambda x: x[1], reverse=True)
global_top_30_items = [item_id for item_id, pop in global_top_items_sorted[:30]]

# 获取测试集用户的历史行为 (正反馈物品)
user_items_dict_test = test_df.groupby('buyer_admin_id')['item_id'].apply(set).to_dict()
all_test_users = test_df['buyer_admin_id'].unique()

submission_data = []

for user_id in tqdm(all_test_users, desc="生成推荐"):
    # 步骤3: 获取用户的正反馈物品列表
    history_items = user_items_dict_test.get(user_id, set())
    
    candidate_scores = defaultdict(float)
    
    # 针对每一个正反馈物品，找出相似物品集合
    for item_i in history_items:
        # 这里为了效率，我们只考虑与item_i最相似的前N个物品
        # 如果不加[:N]，则会考虑所有相似物品
        for item_j, similarity in sorted(item_similarity_matrix.get(item_i, {}).items(), key=lambda x: x[1], reverse=True)[:50]:
            # 过滤掉用户已经买过的
            if item_j not in history_items:
                # 累加相似度分数
                candidate_scores[item_j] += similarity
    
    # 步骤4: 对相似物品集合中的物品，利用相似度分值进行排序
    sorted_candidates = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    recommendations = [item_id for item_id, score in sorted_candidates]
    
    # 回退和补充逻辑
    if len(recommendations) < 30:
        rec_set = set(recommendations)
        for popular_item in global_top_30_items:
            if popular_item not in rec_set and popular_item not in history_items:
                recommendations.append(popular_item)
            if len(recommendations) >= 30:
                break
    
    final_recommendations = recommendations[:30]
    submission_data.append([user_id] + final_recommendations)

# --- 保存提交文件 ---
columns = ['buyer_admin_id'] + [f'predict {i+1}' for i in range(30)]
submission_df = pd.DataFrame(submission_data, columns=columns)
submission_filename = 'classic_item_cf_v2_baseline.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n提交文件 '{submission_filename}' 已成功生成！")

正在加载数据...
正在准备用户购买历史字典...
开始构建物品相似度矩阵 (此过程可能需要较长时间)...


计算余弦相似度: 100%|██████████| 1852415/1852415 [05:33<00:00, 5557.20it/s] 


物品相似度矩阵构建完成。
正在计算全局热度榜单 (用于回退)...
正在为测试用户生成推荐列表...


Generating predictions: 100%|██████████| 10576/10576 [00:55<00:00, 189.01it/s]



提交文件 'classic_item_cf_baseline.csv' 已成功生成！
文件预览:
   buyer_admin_id  predict 1  predict 2  predict 3  predict 4  predict 5  \
0         6664334    3783312    3759092    3756841    3908460    3725537   
1         6394777    7254114    6541314    7173068    4824270    1802691   
2         1746935    3184786    2058901        153     453414     454708   
3         5828988    6127880    5159147      20494    4083696    4338709   
4          421713    6714817    5660680    2432746    6544645    4216322   

   predict 6  predict 7  predict 8  predict 9  ...  predict 21  predict 22  \
0    3696977    3728430    3784302    3699976  ...     6362205     1889135   
1    1223367     839668    5313566    5368075  ...      797410     5444071   
2     713021    6933186    7127253    7026525  ...     4722345     3381598   
3    5098299    7062659    1687813    5319685  ...     1794944     2880647   
4     663714    6579617    1528465     887785  ...     4400785     5212564   

   predict 23  predict 2